In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from tests import multi_param_test as tests

def write_path(instructions):
    path = [(0,0)]
    for bit in instructions:
        d = bit[0]
        mag  = int(bit[1:])
        last = path[-1]
        if d=='R':
            new = (last[0]+mag, last[1])
        elif d=='L':
            new = (last[0]-mag, last[1])
        elif d == 'U':
            new = (last[0], last[1]+mag)
        elif d == 'D':
            new = (last[0], last[1]-mag)
        path.append(new)
    return path


def determine_intersection(ax,ay,bx,by,cx,cy,dx,dy):
    if ax == bx:
        x = ax
        y = cy
        xinterval = [min(cx,dx), max(cx,dx)]
        yinterval = [min(ay,by), max(ay,by)]
        rightx = ax >= xinterval[0] and ax <= xinterval[1]
        righty = cy >= yinterval[0] and cy <= yinterval[1]
        
    else:
        x=cx
        y=ay
        xinterval = [min(ax,bx), max(ax,bx)]
        yinterval = [min(cy,dy), max(cy,dy)]
        rightx = cx >= xinterval[0] and cx <= xinterval[1]
        righty = ay >= yinterval[0] and ay <= yinterval[1]
    if rightx and righty:
        return (x,y)
    else:
        return None


def list_of_intersects(first_desc,second_desc):
    first_wire = write_path(first_desc)
    second_wire = write_path(second_desc)
    intersections = []
    for i in range(len(first_wire)-1):
        for j in range(len(second_wire)-1):
            a = determine_intersection(first_wire[i][0], 
                                               first_wire[i][1], 
                                               first_wire[i+1][0], 
                                               first_wire[i+1][1],
                                              second_wire[j][0],
                                              second_wire[j][1],
                                              second_wire[j+1][0],
                                              second_wire[j+1][1])
            if a:
                intersections.append(a)
    return intersections[1:]

def point_on_line(x1,y1,x2,y2,xn,yn):
    xinterval = [min(x1,x2), max(x1,x2)]
    yinterval = [min(y1,y2), max(y1,y2)]
    return xn >= xinterval[0] and xn <= xinterval[1] and yn >= yinterval[0] and yn <= yinterval[1]


def count_steps(wire_desc,intersect_x,intersect_y):
    wire_path = write_path(wire_desc)
    dist = 0
    verx_1, very_1 = wire_path[0]
    verx_2, very_2 = wire_path[1]
    i = 2
    while not point_on_line(verx_1,very_1, verx_2, very_2,intersect_x,intersect_y):
        dist += abs(verx_1-verx_2)+abs(very_1-very_2)
        verx_1 = verx_2
        very_1 = very_2
        verx_2,very_2 = wire_path[i]
        i+=1
    dist += abs(verx_1-intersect_x)+abs(very_1-intersect_y)
    return dist

def min_intersect_dist(first_desc,second_desc):
    intersections = list_of_intersects(first_desc,second_desc)
    dists = []
    for x,y in intersections:
        dists.append(count_steps(first_desc,x,y)+count_steps(second_desc,x,y))
    return np.min(dists)

In [15]:
ins = [(['R8','U5','L5','D3'],['U7','R6','D4','L4']),
      ("R75,D30,R83,U83,L12,D49,R71,U7,L72".split(','),'U62,R66,U55,R34,D71,R55,D58,R83'.split(",")),
      ("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51".split(","),"U98,R91,D20,R16,D67,R40,U7,R15,U6,R7".split(","))]
outs = [30,610,410]

In [16]:
tests(min_intersect_dist,ins,outs)

True

In [17]:
with open("2019_12_3-input.txt") as fileobj:
    raw = fileobj.read()
paths = raw.split("\n")
min_intersect_dist(paths[0].split(","), paths[1].split(","))

163676